# Get the most ordered custom-products

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Catalog/Custom_products/data'
FILE_NAME = 'custom_products.csv'

## Code

In [16]:
from pathlib import Path

from numpy import nan
from emoji import demojize
from re import sub, UNICODE
from unicodedata import normalize
from pandas import DataFrame, Series, read_csv, cut, to_datetime

from sklearn.feature_extraction.text import CountVectorizer

class CustomProducts:
    def __init__(self, base_dir, file_name) -> None:
        '''
        Inicializa la clase recibiendo un directorio un nombre de archivo
        '''
        # Obtiene un directorio como texto y convertirlo a tipo Path para unir directorios, buscar archivos, etc.
        self.base_dir = Path(base_dir)
        # Guarda el nombre del archivo como atributo
        self.file_name = file_name

        # Concatena el directorio y el nombre del archivo
        self.file_path = self.base_dir.joinpath(self.file_name)
        # Revisa si el archivo existe
        if not self.file_path.is_file(): print(f'It should be a file called "{self.file_name}" at\n{self.base_dir}\n\nBut there is not, add it and try again')


    def two_char(self, n: float) -> str:
        '''
        Función para convertir float: 1.0 --> str: '01'
        '''
        return str(int(n)).zfill(2)


    def create_bins(self, df: DataFrame, col: str, bins: list, lower_limit=-1, upper_limit=1000) -> Series:
        '''
        Recibiendo los cortes, recibe una columna numérica y crea rangos tipo "00", "01 a 05", ">=6"
        '''
        # Crear rangos
        df[f'{col}_range'] = cut(df[col], bins=[lower_limit]+bins+[upper_limit])
        # Convertirlo a texto: [1.0 - 5.0] --> '01 a 05'
        df[f'{col}_range'] = df[f'{col}_range'].map(lambda x: self.two_char(x.left+1)+' a '+self.two_char(x.right) if x!=nan else nan)

        # Corregir algunas etiquetas como: '01 a 01'-->'01' y también '03 a upper_limit'-->'>= 03'
        last_cut = self.two_char(bins[-1]+1)
        df[[f'{col}_range']] = df[[f'{col}_range']].replace({
            **{last_cut+f' a {upper_limit}': '>= '+last_cut},
            **{self.two_char(x)+' a '+self.two_char(x): self.two_char(x) for x in bins}
        })
        # No perder de vista los valores ausentes: "La falta de información también es información"
        df[f'{col}_range'] = df[f'{col}_range'].map(lambda x: nan if str(x)=='nan' else str(x))

        return df[f'{col}_range']


    def date_vars(self, df: DataFrame, date_col: str='date', hours_bin: list=[9,12,14,17,20], **kwargs) -> DataFrame: 
        '''
        Crear variables de fecha: año, trimestre, mes, hora y rangos de hora
        '''
        # Convertir a tipo datetime
        df[date_col] = to_datetime(df[date_col], **kwargs)

        # Para extraer la división de año
        df[f'{date_col}_year'] = df[date_col].dt.year.map(int).map(str)
        # Trimestre a dos caracteres
        df[f'{date_col}_quarter'] = df[date_col].dt.quarter.map(self.two_char)+'Q'
        # Mes
        df[f'{date_col}_month'] = df[date_col].dt.month
        # Semana a dos caracteres
        df[f'{date_col}_week'] = df[date_col].dt.isocalendar().week.map(self.two_char)+'W'

        # Concatenar el año, tanto trimestre como con el mes
        df[f'{date_col}_yearquarter'] = df[f'{date_col}_year']+' - '+df[f'{date_col}_quarter']
        df[f'{date_col}_yearmonth'] = df[f'{date_col}_year']+' - '+df[f'{date_col}_month'].map(self.two_char)+'M'
        df[f'{date_col}_yearweek'] = df[f'{date_col}_year']+' - '+df[f'{date_col}_week']

        # Create the list+dict to map "jul" --> "07_jul"
        list_month = ['ene','feb','mar','abr','may','jun','jul','ago','sep','oct','nov','dic']
        dict_month = dict(zip(
            range(1,13), 
            map(lambda x: x[0]+'_'+x[-1], zip(map(self.two_char, range(1,13)),list_month))
        ))
        df[f'{date_col}_month'] = df[f'{date_col}_month'].map(dict_month)

        # Día de la semana, sólo los primeros 3 caracteres
        df[f'{date_col}_weekday'] = df[date_col].dt.day_name().str[:3]

        # Hora
        df[f'{date_col}_hour'] = df[date_col].dt.hour
        # Crear rangos de hora
        df[f'{date_col}_hour_range'] = self.create_bins(df, f'{date_col}_hour', bins=hours_bin)

        # Mantener sólo la fecha
        df[date_col] = df[date_col].dt.date
        return df

    def clean_text(self, text: str, rem_stop: list, pattern: str="[^a-zA-Z0-9\s\-\/]", lower: bool=True, emoji: bool=True, to_singular: bool=True) -> str: 
        '''
        Limpieza de texto
        '''
        # "Traduce" emojis, ej: 🇲🇽 --> :Mexico:
        if emoji: text = demojize(text)

        # Reemplazar acentos: áàäâã --> a
        clean = normalize('NFD', str(text).replace('\n', ' \n ')).encode('ascii', 'ignore')
        # Omitir caracteres especiales !"#$%&/()=...
        clean = sub(pattern, ' ', clean.decode('utf-8'), flags=UNICODE)

        # Mantener sólo un espacio
        clean = sub(r'\s{2,}', ' ', clean.strip())

        # Minúsculas si el parámetro lo indica
        if lower: clean = clean.lower()
        # Omitir la última "s" si el parámetro lo indica
        if to_singular: clean = sub('s\s',' ',clean+' ')

        # Omitir las stopwords indicadas
        clean = ' '.join([x for x in clean.split() if x not in rem_stop])

        # Si el registro estaba vacío, indicar nulo
        if clean in ('','nan'): clean = nan
        return clean

    def data_wrangling(self, stopw: list, cols_to_split: list=['store','branch','product','brand','category'], cols_to_merge: list=['category','brand','product','custom_request'], custom_req_col: str='custom_request', **kwargs) -> DataFrame:
        # Obtiene el csv
        df = read_csv(self.file_path, **kwargs)

        # Separa cada columna en dos a partir del primer caracter "-"
        for col in cols_to_split: df[[f'{col}_id',col]] = df[col].str.split('-', n=1, expand=True)
        # Crea las variables de fecha
        df = self.date_vars(df, **kwargs)

        # Une las palabras para no asumir por ejemplo la categoría "fruta fresca" como dos palabras sino una
        for col in cols_to_merge: 
            # Con excepción de la columna que indica el custom_request
            if col==custom_req_col: pass
            else: df[col] = df[col].map(lambda x: str(x).strip().replace(' ',''))

        # Une las columnas que se indiquen en el parámetro para limpiar el texto
        df['text'] = df[cols_to_merge].apply(' '.join, axis=1).map(lambda x: self.clean_text(x,rem_stop=stopw))
        return df

    def top_words(self, df: DataFrame, col: str='text', top_n: int=100, **kwargs) -> DataFrame:
        cv = CountVectorizer(max_features=top_n, **kwargs)
        cv_fit = cv.fit_transform(df[col])
        top = dict(zip(cv.get_feature_names(), cv_fit.toarray().sum(axis=0)))
        top = DataFrame(top, index=['word_count']).T.sort_values('word_count', ascending=False)
        return top


cp = CustomProducts(BASE_DIR, FILE_NAME)

In [28]:
custom_stopwords = ['heb','empty','si','no','ni','sin','que','q','un','una','uno','el','la','los','en','de','y','mi','para','por','favor','porfavor','porfa','has','re','esten','muy','ma','lo','se','sea','solo','este','do','con','hay','gracia','pieza']
df = cp.data_wrangling(stopw=custom_stopwords)
df.sample()

,date,order_id,city,zone,store,branch,category,brand,product,custom_request,...,date_quarter,date_month,date_week,date_yearquarter,date_yearmonth,date_yearweek,date_weekday,date_hour,date_hour_range,text
2925,2021-10-23,44068056,Monterrey,Suroeste,HEB,eFC Aaron Saenz,Verdurasyfrutasenvasadas,HEB,Freshmelóncortado,En dos trastes de 300,...,04Q,10_oct,42W,2021 - 04Q,2021 - 10M,2021 - 42W,Sat,20,18 a 20,verdurasyfrutasenvasada freshmeloncortado tras...


In [29]:
cp.top_words(df, top_n=33)

,word_count
frutasfresca,1539
verdurasfresca,1377
verde,556
aguacatehasssupremo,538
platano,529
jamondepavo,370
carnedere,363
maduro,363
rebanada,349
papayamaradol,290
